In [1]:
!pip install faiss-cpu
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 62.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 38.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [2]:
 # ✅ Define your branch name as a string
BRANCH = "yunxiu-branch"

# ✅ Clone your GitHub repo and switch to your branch
!git clone -b $BRANCH https://github.com/xiuxiuface/dsa4213-project.git

# ✅ Move into the project folder
%cd dsa4213-project


Cloning into 'dsa4213-project'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 202 (delta 59), reused 54 (delta 27), pack-reused 88 (from 2)
Receiving objects: 100% (202/202), 84.68 MiB | 40.54 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/kaggle/working/dsa4213-project


In [3]:

# Load Dataset and Create Subset
# ============================================================================
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import json
import os

# Load the ORIGINAL dataset
df_full = pd.read_excel("rag_dataset_clean.xlsx")
print(f"Full dataset shape: {df_full.shape}")

# ============================================================================
# CREATE SMALL SUBSET FOR TESTING (CHANGE THIS NUMBER!)
# ============================================================================
df = df_full.copy()  
print(f"✓ Using full dataset of {len(df)} samples")
print(f"Columns: {df.columns.tolist()}")

# Prepare data
passages = df["passage"].astype(str).tolist()
questions = df["question"].astype(str).tolist()
answers = df["final_decision"].astype(str).tolist()

Full dataset shape: (13738, 4)
✓ Using full dataset of 13738 samples
Columns: ['question', 'answer', 'final_decision', 'passage']


In [4]:
import transformers
print(transformers.__version__)


4.53.3


In [5]:
from sentence_transformers import SentenceTransformer
import faiss
import torch

print("Loading BioBERT model for dense retrieval...")
biobert_model = SentenceTransformer("pritamdeka/S-PubMedBert-MS-MARCO")

print(f"Encoding {len(passages)} passages with BioBERT...")
passage_embeddings = biobert_model.encode(
    passages, 
    show_progress_bar=True,
    batch_size=32
)

print("Building FAISS index on")
dimension = passage_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(passage_embeddings.astype('float32'))
print(f"✓ FAISS index built with {index.ntotal} passages")

def retrieve_dense(query, k=3):
    """Retrieve top-k passages using dense embeddings"""
    query_emb = biobert_model.encode([query])
    distances, indices = index.search(query_emb.astype('float32'), k)
    retrieved_passages = [passages[i] for i in indices[0]]
    return " ".join(retrieved_passages), indices[0]


2025-11-08 08:17:04.863853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762589825.017074      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762589825.062286      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encoding 13738 passages with BioBERT...


Batches:   0%|          | 0/430 [00:00<?, ?it/s]

Building FAISS index on
✓ FAISS index built with 13738 passages


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print("Loading FLAN-T5 for answer generation...")
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(device)
print(f"✓ Model loaded on {device}")

def generate_answer(question, context, max_new_tokens=10):
    """Generate concise yes/no answer for classification"""
    prompt = f"""You are a medical research assistant.
Based on the context below, answer the question with only 'Yes' or 'No'.
Do not include explanations or extra words.

Context: {context}

Question: {question}

Answer (Yes or No):"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = qa_model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return answer


Loading FLAN-T5 for answer generation...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✓ Model loaded on cuda


In [7]:
print("\n" + "="*70)
print("RUNNING ABLATION STUDY")
print("="*70)

# ============================================================================
# REDUCE CONFIGURATIONS FOR FASTER TESTING
# ============================================================================
retrieval_methods = ["Dense"]
top_k_values = [1,3,5]  # ← Just k=3 for now, add [1,3,5] later

all_results = {}

for method in retrieval_methods:
    for k in top_k_values:
        config_name = f"{method}_k{k}"
        print(f"\n{'='*60}")
        print(f"Running: {config_name} on {len(questions)} questions")
        print(f"{'='*60}")
        
        predictions = []
        retrieved_indices = []
        
        retrieve_fn = retrieve_bm25 if method == "BM25" else retrieve_dense
        
        for question in tqdm(questions, desc=f"Generating ({config_name})"):
            context, indices = retrieve_fn(question, k=k)
            answer = generate_answer(question, context)
            predictions.append(answer)
            retrieved_indices.append(indices)
        
        all_results[config_name] = {
            "predictions": predictions,
            "retrieved_indices": retrieved_indices
        }
        
        print(f"✓ Completed {config_name}")



RUNNING ABLATION STUDY

Running: Dense_k1 on 13738 questions


Generating (Dense_k1):   0%|          | 0/13738 [00:00<?, ?it/s]

✓ Completed Dense_k1

Running: Dense_k3 on 13738 questions


Generating (Dense_k3):   0%|          | 0/13738 [00:00<?, ?it/s]

✓ Completed Dense_k3

Running: Dense_k5 on 13738 questions


Generating (Dense_k5):   0%|          | 0/13738 [00:00<?, ?it/s]

✓ Completed Dense_k5


In [8]:
from evaluate import load

print("\n" + "="*70)
print("EVALUATION METRICS")
print("="*70)

squad_metric = load("squad")

"""
def token_f1(pred, ref):
    pred_tokens = set(pred.lower().split())
    ref_tokens = set(ref.lower().split())
    overlap = len(pred_tokens & ref_tokens)
    if not pred_tokens or not ref_tokens: return 0.0
    prec = overlap / len(pred_tokens)
    rec = overlap / len(ref_tokens)
    return 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0
"""
evaluation_results = {}
retrieval_metrics = {}

from sklearn.metrics import precision_score, recall_score, f1_score


for config_name, results in all_results.items():
    preds = [p.strip().lower() for p in results["predictions"]]
    refs = [a.strip().lower() for a in answers]
    
    em = np.mean([1 if p == a else 0 for p, a in zip(preds, refs)])
    
    f1_macro = f1_score(refs, preds, average='macro', labels=['yes','no'], zero_division=0)
    precision_macro = precision_score(refs, preds, average='macro', labels=['yes','no'], zero_division=0)
    recall_macro = recall_score(refs, preds, average='macro', labels=['yes','no'], zero_division=0)
    
    evaluation_results[config_name] = {
        "EM": em,
        "F1_Score": f1_macro, 
        "Precision": precision_macro,
        "Recall": recall_macro,
    }
    
    print(f"\n{config_name}:")
    print(f"  EM={em:.4f}, F1={f1_macro:.4f}, P={precision_macro:.4f}, R={recall_macro:.4f}")


EVALUATION METRICS



Dense_k1:
  EM=0.8418, F1=0.5686, P=0.5584, R=0.6072

Dense_k3:
  EM=0.7768, F1=0.4911, P=0.5043, R=0.5099

Dense_k5:
  EM=0.7768, F1=0.4911, P=0.5043, R=0.5099


In [9]:
for config_name, results in all_results.items():
    retrieved_indices = results["retrieved_indices"]
    precisions, recalls = [], []

    for i, indices in enumerate(retrieved_indices):
        true_idx = i  # assume 1-to-1 match (question i ↔ passage i)
        hit = 1 if true_idx in indices else 0
        precisions.append(hit / len(indices))
        recalls.append(hit)

    retrieval_metrics[config_name] = {
        f"Precision@{len(indices)}": np.mean(precisions),
        f"Recall@{len(indices)}": np.mean(recalls)
    }


In [ ]:
import pandas as pd

results_summary = []
for config_name in all_results.keys():
    method, k_str = config_name.split("_")
    k = int(k_str[1:])
    
    row = {
        "Method": method,
        "Top-k": k,
        "EM": evaluation_results[config_name]["EM"],
        "F1_Score": evaluation_results[config_name]["F1_Score"],  # sklearn F1
        "Precision": evaluation_results[config_name]["Precision"],
        "Recall": evaluation_results[config_name]["Recall"],
        "Precision@k": retrieval_metrics[config_name][f"Precision@{k}"],
        "Recall@k": retrieval_metrics[config_name][f"Recall@{k}"]
    }
    results_summary.append(row)

results_df = pd.DataFrame(results_summary)
results_df.to_csv("results_qa/qa_ablation_study.csv", index=False)

print("\n✓ Ablation results saved to: results/qa_ablation_study.csv")
print(results_df)
print("\n" + "="*60)
print("FULL-DATASET ABLATION RESULTS (Yes/No)")
print("="*60)
print(results_df.to_string(index=False))



✓ Ablation results saved to: results/qa_ablation_study.csv
  Method  Top-k        EM  F1_Score  Precision    Recall  Precision@k  \
0  Dense      1  0.841753  0.568554   0.558404  0.607241     0.859878   
1  Dense      3  0.776823  0.491088   0.504261  0.509939     0.315111   
2  Dense      5  0.776823  0.491088   0.504261  0.509939     0.192939   

   Recall@k  
0  0.859878  
1  0.945334  
2  0.964696  

FULL-DATASET ABLATION RESULTS (Yes/No)
Method  Top-k       EM  F1_Score  Precision   Recall  Precision@k  Recall@k
 Dense      1 0.841753  0.568554   0.558404 0.607241     0.859878  0.859878
 Dense      3 0.776823  0.491088   0.504261 0.509939     0.315111  0.945334
 Dense      5 0.776823  0.491088   0.504261 0.509939     0.192939  0.964696


In [ ]:
import os, json


output_df = df.copy()
for config_name, results in all_results.items():
    output_df[f"pred_{config_name}"] = results["predictions"]
output_df.to_csv("results/rag_qa_full_outputs.csv", index=False)

with open("results/rag_qa_full_metrics.json", "w") as f:
    json.dump({
        "evaluation_results": evaluation_results,
        "retrieval_metrics": retrieval_metrics
    }, f, indent=2)

print("\n✓ Results saved to:")
print("  - results/rag_qa_full_outputs.csv")
print("  - results/rag_qa_full_metrics.json")




✓ Results saved to:
  - results/rag_qa_full_outputs.csv
  - results/rag_qa_full_metrics.json
